# Crear el dataset sobre personoas pensionadas en méxico

## Vamos a importar las librerias que sean necesarias para crear el conjnto de datos del ISSTE

In [1]:
import requests
import pandas as pd
from io import StringIO
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


## Definimos la función para obtener los datos de la página (opcional descargalos)

In [2]:
def bajar_datos():  
    url = "https://www.issste.gob.mx/datosabiertos/pensionadosdirectos.txt"

    response = requests.get(url)
    response.encoding = 'utf-8'

    data = StringIO(response.text)
    df = pd.read_csv(data, sep="\t", dtype=str) 
    
    # df.to_csv('isste_data.csv', index=False)
    return df

## La función principal

In [ ]:
if __name__ == "__main__":
    #df = bajar_datos()
    df = pd.read_csv('isste_data.csv', sep="\t", dtype=str)


In [6]:
df = pd.read_csv('isste_data.csv')

In [7]:
df.columns

Index(['id', 'anio', 'mes', 'nombre', 'paterno', 'materno', 'sexo',
       'primer_ramo', 'primer_ramo_des', 'primera_fecha', 'ult_ramo',
       'ult_ramo_des', 'ult_fecha', 'tiempo_serv_trab', 'tiempo_serv',
       'ent_pen', 'ent_pen_des', 'tip_pen_org', 'tip_pen_org_des',
       'porcentaje', 'tip_pen_act', 'tip_pen_act_des', 'fecha_inidi',
       'alta_nomina', 'ent_pago', 'ent_pago_des', 'importe_pen', 'importe_msm',
       'neto_pagar'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1019781 entries, 0 to 1019780
Data columns (total 29 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1019781 non-null  int64  
 1   anio              1019781 non-null  int64  
 2   mes               1019781 non-null  int64  
 3   nombre            1019781 non-null  object 
 4   paterno           1019781 non-null  object 
 5   materno           1019521 non-null  object 
 6   sexo              1019781 non-null  object 
 7   primer_ramo       1019766 non-null  float64
 8   primer_ramo_des   1019637 non-null  object 
 9   primera_fecha     1019758 non-null  object 
 10  ult_ramo          1019781 non-null  int64  
 11  ult_ramo_des      1016608 non-null  object 
 12  ult_fecha         1019750 non-null  object 
 13  tiempo_serv_trab  1019781 non-null  int64  
 14  tiempo_serv       1019781 non-null  int64  
 15  ent_pen           1019781 non-null  int64  
 16  

In [12]:
df.sort_values(by='anio')

,id,anio,mes,nombre,paterno,materno,sexo,primer_ramo,primer_ramo_des,primera_fecha,...,porcentaje,tip_pen_act,tip_pen_act_des,fecha_inidi,alta_nomina,ent_pago,ent_pago_des,importe_pen,importe_msm,neto_pagar
1019768,137555851,2025,3,DELIA,PERAZA,OSUNA,MUJER,11.0,SECRETARIA DE EDUCACION PUBLICA (SEP),1982-01-01 00:00:00,...,85.0,102,EDAD Y TIEMPO DE SERVICIOS,2009-01-16 00:00:00,2009-06-01 00:00:00,25,DELEGACION ESTATAL EN SINALOA,20775.0,2,11802.17
1019769,137555852,2025,3,GUADALUPE,GUTIERREZ,BARRAZA,MUJER,11.0,SECRETARIA DE EDUCACION PUBLICA (SEP),1980-10-16 00:00:00,...,100.0,101,JUBILACION,2009-04-01 00:00:00,2009-08-01 00:00:00,8,DELEGACION ESTATAL EN CHIHUAHUA,15299.1,1,14791.38
1019770,137555853,2025,3,CONCEPCION,SALES,LAGUNAS,MUJER,11.0,SECRETARIA DE EDUCACION PUBLICA (SEP),1980-09-01 00:00:00,...,100.0,101,JUBILACION,2010-01-01 00:00:00,2010-03-01 00:00:00,33,DELEGACION EN LA ZONA NORTE DEL D.F.,33942.0,4,33444.41
1019771,137555854,2025,3,ARTEMIO,ZARATE,ANTONIO,HOMBRE,8.0,"SECRETARIA DE AGRICULTURA, GANADERIA, DESARRO...",1979-07-01 00:00:00,...,100.0,101,JUBILACION,2009-12-16 00:00:00,2010-04-01 00:00:00,20,DELEGACION ESTATAL EN OAXACA,13489.8,1,13615.37
1019772,137555855,2025,3,MARIA DEL PILAR,PALOMEQUE,RAMIREZ,MUJER,12909.0,SERVICIOS DE SALUD PUBLICA DEL DISTRITO FEDERAL,1983-03-01 00:00:00,...,100.0,101,JUBILACION,2011-04-01 00:00:00,2011-08-01 00:00:00,35,DELEGACION EN LA ZONA ORIENTE DEL D.F.,24735.6,2,20076.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019777,137555860,2025,3,JOEL,VILLARREAL,VILLARREAL,HOMBRE,8.0,"SECRETARIA DE AGRICULTURA, GANADERIA, DESARRO...",1975-09-16 00:00:00,...,52.5,102,EDAD Y TIEMPO DE SERVICIOS,2005-11-20 00:00:00,2008-11-01 00:00:00,19,DELEGACION ESTATAL EN NUEVO LEON,8364.0,1,8584.00
1019778,137555861,2025,3,MARIA DE LOURDES,DEL VALLE,TREJO,MUJER,11.0,SECRETARIA DE EDUCACION PUBLICA (SEP),1981-09-01 00:00:00,...,100.0,101,JUBILACION,2010-01-01 00:00:00,2010-05-01 00:00:00,35,DELEGACION EN LA ZONA ORIENTE DEL D.F.,9126.6,1,6527.73
1019779,137555862,2025,3,ELPIDIA,CORTES,SANCHEZ,MUJER,11916.0,SECRETARIA DE EDUCACION EN EL ESTADO DE MICHOACAN,1956-03-01 00:00:00,...,100.0,101,JUBILACION,2001-01-01 00:00:00,2001-03-01 00:00:00,16,DELEGACION ESTATAL EN MICHOACAN,29050.2,3,29066.85
1019780,137555863,2025,3,ISAIAS,VANEGAS,MARTINEZ,HOMBRE,11.0,SECRETARIA DE EDUCACION PUBLICA (SEP),1972-02-16 00:00:00,...,100.0,101,JUBILACION,2004-05-16 00:00:00,2004-12-01 00:00:00,34,DELEGACION EN LA ZONA SUR DEL D.F.,12779.1,1,11206.25
